In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import pandas as pd

mnist_pd = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')


In [ ]:
import numpy as np
from IPython.display import display

img_index = 0
img = Image.fromarray(mnist_pd.iloc[img_index, 1:].to_numpy(dtype=np.uint8).reshape((28, 28)))
display(img)
print(mnist_pd.iloc[img_index][0])

1


In [ ]:
mnist = mnist_pd.to_numpy(dtype=np.uint8)
np.random.shuffle(mnist)
train_mnist = mnist[0: 30000]
test_mnist = mnist[30000: 42000]
test_label = test_mnist[:, 0]
test_data = test_mnist[:, 1:].astype(dtype=np.float64)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

train_size = 30000
batch_size = 100
epochs = 10
lr = 0.01
momentum = 0.5


class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 512).double()
    self.fc2 = nn.Linear(512, 256).double()
    self.fc3 = nn.Linear(256, 128).double()
    self.fc4 = nn.Linear(128, 64).double()
    self.fc5 = nn.Linear(64, 32).double()
    self.fc6 = nn.Linear(32, 10).double()

  def forward(self, x):
    h1 = F.relu(self.fc1(x)).double()
    h2 = F.relu(self.fc2(h1)).double()
    h3 = F.relu(self.fc3(h2)).double()
    h4 = F.relu(self.fc4(h3)).double()
    h5 = F.relu(self.fc5(h4)).double()
    h6 = self.fc6(h5).double()
    return F.log_softmax(h6, dim=1).double()

model = Net()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


In [ ]:
def train_one_epoch():
  np.random.shuffle(train_mnist)
  train_label = train_mnist[:, 0]
  train_data = train_mnist[:, 1:]
  for i in range(0, train_size // batch_size):
    optimizer.zero_grad()
    output = model(torch.tensor(train_data[i * batch_size: (i + 1) * batch_size], dtype=torch.float64)).double()
    loss = F.nll_loss(output, torch.tensor(train_label[i * batch_size: (i + 1) * batch_size], dtype=torch.int64)).double()
    loss.backward()
    optimizer.step()

In [ ]:
model.train()
for i in range(0, epochs):
  train_one_epoch()

In [ ]:
model.eval()
output = model(torch.tensor(test_data.astype(np.float64)))
correct = 0
for i in range(12000):
  correct += torch.argmax(output[i]) == test_label[i]

print(correct / 12000 * 100)

tensor(97.3833)


In [ ]:
for img_index in range(0, 10):
  img = Image.fromarray(test_data[img_index].astype(np.uint8).reshape((28, 28)))
  display(img)
  print(test_label[img_index])

2


2


6


0


5


5


7


8


5


4


float64
